In [27]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import glob

In [28]:
CONFIG_PATH = "/home/office5/ATE/Ketan_sir/"
def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")
config

{'l_thresh': 25,
 'u_thresh': 245,
 't_lower': 100,
 't_upper': 200,
 'L2Gradient': True,
 'connectivity': 4,
 'get_pixel_img1': 5,
 'get_pixel_img2': 10,
 'get_kernel_size_value': 1,
 'debug': False,
 'y_size': 200,
 'x_size': 500}

In [29]:
def show(img):
    plt.figure(figsize = (15,12))
    plt.imshow(img)
    plt.show()

In [30]:
def get_edge_image(img1):
    thresh = cv2.threshold(img1,config['l_thresh'],config['u_thresh'],cv2.THRESH_BINARY)[1]
    edge = cv2.Canny(thresh, config['t_lower'], config['t_upper'], L2gradient = config['L2Gradient'])
    return edge

def get_connected_comp_img(edge):
    lst_component = []
    output = cv2.connectedComponentsWithStats(edge, config['connectivity'], cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output

    # loop over the number of unique connected component labels
    for i in range(0, numLabels):
        if i == 0:
            text = "examining component {}/{} (background)".format(
                i + 1, numLabels)
        else:
            text = "examining component {}/{}".format( i + 1, numLabels)
        w_ = stats[i, cv2.CC_STAT_WIDTH]
        h_ = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

        keepWidth = w_ > 2
        keepHeight = h_ > 2
        keepArea = area > 50

        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 255
            color_img  = cv2.cvtColor(componentMask,cv2.COLOR_GRAY2BGR)
            lst_component.append(color_img)
    return lst_component

In [41]:
def get_extreme_point(all_component_image):

    extr_point = []

    for i in range(1,len(all_component_image)):
        gray = cv2.cvtColor(all_component_image[i],cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        # print(coords)
        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        points = []
            
        # for i in range(len(coords)):
        #     if(x_min == coords[i][1]):
        #         points.append([coords[i][0], coords[i][1]]) 
        #         break
        #     else:
        #         pass
        points.append([coords[0][0] , coords[0][1]])
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)

        for i in range(2):
            extr_point.append([points[i][0], points[i][1]])

    return extr_point
        


In [42]:
def set_kernel(ori_gray, extreme_point_arr, set_kernel_value):

    y1_s = extreme_point_arr[0] - config['get_kernel_size_value']
    y2_s = extreme_point_arr[0] + config['get_kernel_size_value']
    x1_s = extreme_point_arr[1] - config['get_kernel_size_value']
    x2_s = extreme_point_arr[1] + config['get_kernel_size_value']

    try:
        kernel_value =  ori_gray[y1_s:y2_s + 1,x1_s:x2_s + 1]
        
        for i in range(len(kernel_value[0])):
            for j in range(len(kernel_value[1])):
                set_kernel_value[i][j] = (np.subtract(int(kernel_value[config['get_kernel_size_value']][config['get_kernel_size_value']]),int(kernel_value[i][j])))
                set_kernel_value[config['get_kernel_size_value']][config['get_kernel_size_value']] = 5 
                if(set_kernel_value[i][j]==0):
                    set_kernel_value[i][j] = 1
                elif(set_kernel_value[i][j]<0):
                    set_kernel_value[i][j] = 0
                else:
                    set_kernel_value[i][j] = 1
    except:
        set_kernel_value = np.zeros((3,3))
    return set_kernel_value

In [43]:
def get_removed_extreme_point(ori_gray, all_extreme_point):

    final_extreme_point_list = []
    dict1 = {}
    kernel_shape = (config['get_kernel_size_value'] * 2) + 1 ## 1->3 2->5 3->7 kerenl_size
    m = 0
    dict1 = {}
    for i in range(len(all_extreme_point)):
        extreme_point_arr = np.array(all_extreme_point[i])
        zero_kernel_arr = np.zeros((kernel_shape,kernel_shape))
        kernel_value = set_kernel(ori_gray,extreme_point_arr,zero_kernel_arr)
        unique_1_0 = np.unique(kernel_value,return_counts=True)
        if len(unique_1_0[0])==3:
            count_0 = unique_1_0[1][0]
            count_1 = unique_1_0[1][1]
            dict1['0'] = count_0
            dict1['1'] = count_1
            if (count_0==3 and count_1==5) or (count_0==5 and count_1==3):
                corner_1 = kernel_value[0][0]
                corner_2 = kernel_value[0][kernel_shape-1]
                corner_3 = kernel_value[kernel_shape-1][0]
                corner_4 = kernel_value[kernel_shape-1][kernel_shape-1]
                try:
                    corner_point = np.array([corner_1,corner_2,corner_3,corner_4])
                    for key, value in dict1.items():
                        if value == 3:
                            corner_index = np.where(corner_point==int(key)) 

                            if corner_index[0][0] == 0:
                                x,y = 0,0
                                if (kernel_value[x][y]==int(key) and kernel_value[x+1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                    final_extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                            elif corner_index[0][0] == 1:
                                x,y = 0,kernel_shape-1
                                if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x+1][y]==int(key)):
                                    final_extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                            elif corner_index[0][0] == 2:
                                x,y = kernel_shape-1,0
                                if (kernel_value[x][y]==int(key) and kernel_value[x-1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                    final_extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                            else:
                                x,y = kernel_shape-1,kernel_shape-1
                                if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x-1][y]==int(key)):
                                    final_extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])
                except:
                    pass

    return final_extreme_point_list

### 2nd function
making function to get point from original_image, defective image

In [44]:
def get_point_on_image(first_img_point,img1_gray_pth,img2_gray_pth,debug=False):

    # img1_gray_pth = img1_gray_pth
    # img2_gray_pth = img2_gray_pth

    second_img_point = []
    
    for i in range(len(first_img_point)):
        ori_img_loc_y = first_img_point[i][0]
        ori_img_loc_x = first_img_point[i][1]
        x1_f = ori_img_loc_x - config['get_pixel_img1']
        x2_f = ori_img_loc_x + config['get_pixel_img1']
        y1_f = ori_img_loc_y - config['get_pixel_img1']
        y2_f = ori_img_loc_y + config['get_pixel_img1']

        img_crop1  = img1_gray_pth[y1_f:y2_f,x1_f:x2_f]
        img_first = img_crop1.astype(np.int8)

        x1_s = ori_img_loc_x - config['get_pixel_img2']
        x2_s = ori_img_loc_x + config['get_pixel_img2']
        y1_s = ori_img_loc_y - config['get_pixel_img2']
        y2_s = ori_img_loc_y + config['get_pixel_img2']

        img_crop2  = img2_gray_pth[y1_s:y2_s,x1_s:x2_s]
        img_snd = img_crop2.astype(np.int8)
        try:
            dict1 = {}
            for i in range(config['get_pixel_img2']):
                for j in range(config['get_pixel_img2']):
                    dict1[i,j] = np.mean(np.abs(img_first  - img_snd[i:i+config['get_pixel_img2'],j:j+config['get_pixel_img2']]))
            i,j = min(dict1, key=dict1.get)

            s_img_extr_point_x = x1_s + config['get_pixel_img2'] - config['get_pixel_img1'] + j # 10 for each block of first image and -5 for adding from width and height of first image
            s_img_extr_point_y =y1_s + config['get_pixel_img2'] - config['get_pixel_img1'] + i
            second_img_point.append([s_img_extr_point_y,s_img_extr_point_x])
        except:
            pass
        
    if debug:
        print("in debug")
        for i in range(len(first_img_point)):
            img1_gray_pth[first_img_point[i][0]][first_img_point[i][1]] = 255

        for i in range(len(second_img_point)):
            img2_gray_pth[second_img_point[i][0]][second_img_point[i][1]] = 255
        
        # cv2.imwrite('3.png',img2_gray_pth)
        return second_img_point,img1_gray_pth,img2_gray_pth
    else:
        return second_img_point


In [45]:
def get_point_for_both_img(gray_img1):

    ### first function part
    edge = get_edge_image(gray_img1)
    all_component_image = get_connected_comp_img(edge)
    all_extreme_point =  get_extreme_point(all_component_image)
    remove_duplicate_extreme_point = [tuple(row) for row in all_extreme_point]
    remove_duplicate_extreme_point = np.unique(remove_duplicate_extreme_point,axis=0)
    first_img_extreme_point = get_removed_extreme_point(gray_img1,remove_duplicate_extreme_point)
    if first_img_extreme_point:
        return first_img_extreme_point[0]
    else:
        return first_img_extreme_point

In [46]:
def makegrid_img(original_img):
    imgheight=original_img.shape[0]
    imgwidth=original_img.shape[1]
    y1 = 0
    M = imgheight//3
    N = imgwidth//3
    dict1 = {}
    i=0
    
    for y in range(0,imgheight-10,M):
        for x in range(0, imgwidth-10, N):
            i+=1
            y1 = y + M
            x1 = x + N
            original_img_grid = original_img[y:y+M,x:x+N]

            dict1[y,x] = original_img_grid

    # print("len(dict1)",len(dict1))
    return dict1

## Getting four images pixel point from original and defective images

In [47]:
def get_four_image(original_img):
    height , width = original_img.shape
    half_hei, half_wid = height // 2 , width //2
    original_final_point_for_all_images = []
    original_pixel_value = []
    
    ### Final changes needed here for getting fine pixel value
    def get_final_image(ori_img1,first_img_flag=True, block_num=3):
        while first_img_flag:
            block_num+=1
            if (block_num == 9):
                block_num = 0

            original_img_dict = makegrid_img(ori_img1)
            first_img_extreme_point = get_point_for_both_img(list(original_img_dict.items())[block_num][1])
            
            if block_num==3 or first_img_extreme_point:
                # print("block", block_num)

                original_final_point_for_all_images.append(first_img_extreme_point)
                original_pixel_value.append(list(original_img_dict.items())[block_num][0])
                
                # print("first_img_extreme_point",first_img_extreme_point)
                # print("Break")
                break
            else: 
                pass
                
        first_img_flag = True

        return original_pixel_value


    def get_original_img_point(original_final_point_for_all_images, original_pixel_value, half_hei, half_wid):
        l = []
        m = []
        for i in range(len(original_final_point_for_all_images)):
            if i==0:
                f = np.array(original_final_point_for_all_images[i]) + np.array(original_pixel_value[i]) + np.array((0, 0))
                l.append(list(f))
            elif i==1:
                f = np.array(original_final_point_for_all_images[i]) + np.array(original_pixel_value[i]) + np.array((0, half_wid))
                l.append(list(f))
            elif i==2:
                f = np.array(original_final_point_for_all_images[i]) + np.array(original_pixel_value[i]) + np.array((half_hei, 0))
                l.append(list(f))
            elif i==3:
                f = np.array(original_final_point_for_all_images[i]) + np.array(original_pixel_value[i]) + np.array((half_hei, half_wid))
                l.append(list(f))
        
        return l


        # for first block of original image
    img1_y1, img1_y2, img1_x1, img1_x2 = 0 , half_hei, 0 , half_wid
    ori_img1 = original_img[img1_y1:img1_y2, img1_x1:img1_x2]
    first_img_val = get_final_image(ori_img1)

    # for second block of original image
    img2_y1, img2_y2, img2_x1, img2_x2 = 0 ,  half_hei , half_wid ,  width
    ori_img2 = original_img[img2_y1:img2_y2, img2_x1:img2_x2]
    second_img_val = get_final_image(ori_img2)
    
    # for third block of original image
    img3_y1, img3_y2, img3_x1, img3_x2 = half_hei, height , 0 , half_wid
    ori_img3 = original_img[img3_y1:img3_y2, img3_x1:img3_x2]
    third_img_val = get_final_image(ori_img3)

    # for fourth block of original image
    img4_y1, img4_y2, img4_x1, img4_x2 = half_hei, height , half_wid, width
    ori_img4 = original_img[img4_y1:img4_y2, img4_x1:img4_x2]
    fourth_img_val = get_final_image(ori_img4)
    

    original_point = get_original_img_point(original_final_point_for_all_images, original_pixel_value, half_hei, half_wid)

    return original_point

### Calling as per piyush sir request

In [48]:
## reading image
original_img = cv2.imread('/home/office5/Downloads/01.bmp',0)
defective_img = cv2.imread('/home/office5/Downloads/03.bmp',0)

In [49]:
### to get four point from original image
original_point_list = get_four_image(original_img)
original_point_list

[[553, 1560], [803, 5976], [2128, 1761], [2390, 5976]]

In [50]:
# function to get extreme point from second point same as original extreme point
defective_point_list = get_point_on_image(original_point_list,original_img,defective_img)
defective_point_list

[[553, 1561], [803, 5977], [2128, 1762], [2390, 5977]]

In [51]:
# for drawing original extreme point on original image
for i in range(len(original_point_list)):
    original_img[original_point_list[i][0]][original_point_list[i][1]] = 255

for j in range(len(defective_point_list)):
    defective_img[defective_point_list[j][0]][defective_point_list[j][1]] = 255

cv2.imwrite('original_img_point.png',original_img)
cv2.imwrite('defective_img_point.png',defective_img)

True